In [9]:
import torchaudio
from speechbrain.pretrained import HIFIGAN
from speechbrain.lobes.models.FastSpeech2 import mel_spectogram

# Load a pretrained HIFIGAN Vocoder
hifi_gan = HIFIGAN.from_hparams(source="speechbrain/tts-hifigan-libritts-16kHz", savedir="vocoder_16khz", run_opts={"device":"cuda"})

# Load an audio file (an example file can be found in this repository)
# Ensure that the audio signal is sampled at 16000 Hz
signal, rate = torchaudio.load(r'C:\Users\JadHa\Desktop\Uni\Audio SP\VoiceConversion\VCTK-Corpus-0.92\wav48_silence_trimmed\p225\p225_003_mic1.flac')
resampler = torchaudio.transforms.Resample(orig_freq=rate, new_freq=16000)
signal = resampler(signal)
print(signal.shape)

# Ensure the audio is single channel
signal = signal[0].squeeze()

torchaudio.save('waveform.wav', signal.unsqueeze(0).cpu(), 16000)

# Compute the mel spectrogram.
# IMPORTANT: Use these specific parameters to match the Vocoder's training settings for optimal results.
spectrogram, _ = mel_spectogram(
    audio=signal.squeeze(),
    sample_rate=16000,
    hop_length=256,
    win_length=1024,
    n_mels=80,
    n_fft=1024,
    f_min=0.0,
    f_max=8000.0,
    power=1,
    normalized=False,
    min_max_energy_norm=True,
    norm="slaney",
    mel_scale="slaney",
    compression=True
)

print(spectrogram.shape)

# Convert the spectrogram to waveform
waveforms = hifi_gan.decode_batch(spectrogram)

# Save the reconstructed audio as a waveform
torchaudio.save('waveform_reconstructed.wav', waveforms.squeeze(1).cpu(), 16000)

# If everything is set up correctly, the original and reconstructed audio should be nearly indistinguishable

torch.Size([1, 121411])
torch.Size([80, 475])
